In [42]:
with open('/nas/home/gujiashe/kb/sentences_birth.tsv', 'r') as t:
    i = 0
    for line in t:
        i+=1
    print(i)

21323


In [43]:
import datetime
import csv
with open('/data02/wikidata-frame-completion/data/birth_table.tsv', 'r') as t:
    i = 0
    sentences_birth = open("sentences_birth.tsv", "w")
    tsv_writer = csv.writer(sentences_birth, delimiter='\t')
    for line in t:
        i+=1
        if i==1:
            continue
        
        person, mother, place, date = line.split('\t')
        if len(mother) == 0:
            continue
        person = person.split("@")[0].strip("''").split()
        mother = mother.split("@")[0].strip("''").split()
        place = place.split("@")[0].strip("''").split()
        # print("person: ", person,  "mother: ", mother, "place: ", place, "date: ", date, len(place), len(date))
        # s = "[PERSON] was born on [DATE_OF_BIRTH] at [PLACE_OF_BIRTH]. His mother is [MOTHER]."
        s = person.copy()
        s.extend(["was", "born"])
        spans = []
        len_place = len(place)
        len_date = len(date)
        if len_place == 0 and len_date < 2:
            continue
        if len_place > 0:
            s.append("at")
            s.extend(place)
            spans.append([len(s) - len(place), len(s) - 1])
        if len_date > 1:
            date = date.split('T')[0][1:]
            year, month, day = date.split('-')
            year = int(year)
            month = int(month)
            day = int(day)
            d = datetime.date(year, month, day)
            date = d.strftime("%d %B %Y").split()
            s.append("on")
            s.extend(date)
            spans.append([len(s) - 3, len(s) - 1])
        s.append('.')
        if len(mother) > 0:
            s.extend(["His", "mother", "is"])
            s.extend(mother)
            spans.append([len(s) - len(mother), len(s) - 1])
        # print(s)
        for span in spans:
            tsv_writer.writerow(span + s)
        s = " ".join(s)
        # print(spans)
        # print(s)
        # if i>10:
        #     breaks
    sentences_birth.close()
    print(i)

141132


In [ ]:
# kgtk query -i Person0.tsv  -i Mother0.tsv  -i Place0.tsv  -i Date0.tsv  -i labels.en.tsv.gz --match "Person: (q)-[]->(n)" --opt "Mother: (q)-[]->(m), labels: (m)-[]->(mm)" --opt "Place: (q)-[]->(p), labels: (p)-[]->(pp)" --opt "Date: (q)-[]->(d)" --return "n as Person, mm as Mother, pp as Place_of_birth, d as Date_of_birth" -o birth_table.tsv